In [6]:
def standard_deviation (weights, cov_matrix):
    variance = weights.T @ cov_matrix @ weights
    return np.sqrt(variance)

In [7]:
def expected_returns (weights, log_returns):
    return np.sum(log_returns.mean()*weights)*252

In [8]:
def sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return(expected_returns (weights, log_returns)- risk_free_rate) / standard_deviation(weights, cov_matrix)

In [9]:
def neg_sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return -sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate)

In [10]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from scipy.optimize import minimize
import numpy as np
from fredapi import Fred

selected_tickers = ["AAPL", "GOOG", "TSLA"]
end_date = datetime.today()
start_date = end_date - timedelta(days = 5 * 365)
adj_close_df = yf.download(selected_tickers, start=start_date, end=end_date)['Adj Close']
adj_close_df

[*********************100%%**********************]  3 of 3 completed


Ticker,AAPL,GOOG,TSLA
Date,,,
2019-03-04,42.322655,57.389999,19.024000
2019-03-05,42.245632,58.101501,18.436001
2019-03-06,42.002560,57.893002,18.416000
2019-03-07,41.516392,57.165001,18.439333
2019-03-08,41.615070,57.116001,18.942667
...,...,...,...
2024-02-23,182.520004,145.289993,191.970001
2024-02-26,181.160004,138.750000,199.399994
2024-02-27,182.630005,140.100006,199.729996


In [11]:
log_returns = np.log(adj_close_df/adj_close_df.shift(1))
log_returns = log_returns.dropna()
log_returns

Ticker,AAPL,GOOG,TSLA
Date,,,
2019-03-05,-0.001822,0.012321,-0.031396
2019-03-06,-0.005770,-0.003595,-0.001085
2019-03-07,-0.011642,-0.012655,0.001266
2019-03-08,0.002374,-0.000858,0.026931
2019-03-11,0.034056,0.028853,0.023581
...,...,...,...
2024-02-23,-0.010085,-0.000207,-0.027944
2024-02-26,-0.007479,-0.046058,0.037974
2024-02-27,0.008082,0.009683,0.001654


In [12]:
cov_matrix = log_returns.cov()*252
fred = Fred(api_key='8e01919c368c8b27138da997db27d9ff')
ten_year_treasury_rate = fred.get_series_latest_release('GS10')/100
risk_free_rate = ten_year_treasury_rate.iloc[-1]
risk_free_rate

0.0406

In [13]:
constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights)-1}
bounds = [(0,0.5) for _ in range(len(selected_tickers))]
initial_weights = np.array([1/len(selected_tickers)]*len(selected_tickers))
optimise_results = minimize(neg_sharpe_ratio, initial_weights, args=(log_returns, cov_matrix, risk_free_rate), method='SLSQP', constraints=constraints, bounds=bounds)
optimal_weights = optimise_results.x
optimal_portfolio_return = expected_returns(optimal_weights, log_returns)
optimal_portfolio_volatility = standard_deviation(optimal_weights, cov_matrix)
optimal_sharpe_ratio = sharpe_ratio(optimal_weights, log_returns, cov_matrix, risk_free_rate)